<a href="https://colab.research.google.com/github/talamram/NLP/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [21]:
!git clone https://github.com/NLP-Reichman/2025_assignment_1.git
!mv 2025_assignment_1/data data
!rm 2025_assignment_1/ -r

Cloning into '2025_assignment_1'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 71 (delta 24), reused 44 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 7.05 MiB | 21.82 MiB/s, done.
Resolving deltas: 100% (24/24), done.
mv: cannot move '2025_assignment_1/data' to 'data/data': Directory not empty


# Introduction
In this assignment you will be creating tools for learning and testing language models. The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.
The relevant files are under the data folder:

- en.csv (or the equivalent JSON file)
- es.csv (or the equivalent JSON file)
- fr.csv (or the equivalent JSON file)
- in.csv (or the equivalent JSON file)
- it.csv (or the equivalent JSON file)
- nl.csv (or the equivalent JSON file)
- pt.csv (or the equivalent JSON file)
- tl.csv (or the equivalent JSON file)

In [22]:
import os
import json
from google.colab import files
import pandas as pd
import numpy as np
from itertools import product

# Implementation

## Part 1
Implement the function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. Our token definition is a single UTF-8 encoded character. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data. The vocabulary should include the `<start>` and  `<end>` tokens.

Note - do NOT lowecase the sentences.

In [23]:
import csv
def preprocess() -> list[str]:
  '''
  Return a list of characters, representing the shared vocabulary of all languages
  '''
  vocab = []
  ########################################
  seen = set()
  csv_files = ['data/en.csv',
                'data/es.csv',
                'data/fr.csv',
                'data/in.csv',
                'data/it.csv',
                'data/nl.csv',
                'data/pt.csv',
                'data/tl.csv']

  vocab = ['<start>', '<end>']
  for path in csv_files:
      with open(path, 'r', newline='', encoding='utf-8') as csv_file:
          reader = csv.reader(csv_file)
          next(reader, None)
          for row in reader:
              for ch in row[1]:
                  if ch not in seen:
                      vocab.append(ch)
                      seen.add(ch)

  ########################################
  return vocab

In [24]:
# vocab = preprocess()
# print(f"vocab length: {len(vocab)}")
# print(f"Some characters in the vocab: {vocab[:10]}")

## Part 2
Implement the function *build_lm* that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant *n*-1 sequences, and the values are dictionaries with the *n*_th tokens and their corresponding probabilities to occur. To ensure consistent probabilities calculation, please add n-1 `<start>` tokens to the beginning of a tweet and one `<end>` token at the end. For example, for a trigram model (tokens are characters), it should look something like:

{ "ab":{"c":0.5, "b":0.25, "d":0.25}, "ca":{"a":0.2, "b":0.7, "d":0.1} }

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add-one smoothing information to the dictionary and implement it.

Please add the `<unk>` token with $p(<unk>)=1/|V|$ to the LM if buiulding a smoothed LM.

In [25]:
import math
from collections import deque
def build_lm(lang: str, n: int, smoothed: bool = False) -> dict[str, dict[str, float]]:
  '''
  Return a language model for the given lang and n_gram (n)
  :param lang: the language of the model
  :param n: the n_gram value
  :param smoothed: boolean indicating whether to apply smoothing
  :return: a dictionary where the keys are n_grams and the values are dictionaries
  '''
  LM = {}
  ########################################
  vocab = preprocess()
  vocab_len = len(vocab)
  max_size = n - 1

  csv_path = f'data/{lang}.csv'
  with open(csv_path, 'r', newline='', encoding='utf-8') as csv_file:
      reader = csv.reader(csv_file)
      next(reader, None)
      for row in reader:
          q = deque(['<start>'] * max_size, maxlen=max_size)
          chars = list(row[1])
          chars.append('<end>')
          for ch in chars:
              key = ''.join(str(x) for x in q)
              value = ch
              if key in LM:
                  if value in LM[key]:
                      LM[key][value] = LM[key][value] + 1
                  else:
                      LM[key][value] = 1
              else:
                  LM[key] = {}
                  LM[key][value] = 1
              q.append(ch)

  # normalize the probabilities
  if smoothed:
      for ngram in LM:
          total_count = sum(LM[ngram].values())
          for token in LM[ngram]:
              LM[ngram][token] += 1
              LM[ngram][token] /= (total_count + vocab_len)
      LM['<unk>'] = {v : 1 / vocab_len for v in vocab}
  else:
      for ngram in LM:
          total_count = sum(LM[ngram].values())
          for token in LM[ngram]:
              LM[ngram][token] /= total_count
  ########################################
  return LM

In [26]:
# LM = build_lm("en", 3, False)
# print(f"English Language Model with 3-gram is of length: {len(LM)}")

In [27]:
# # TO DELETEEEEEEE

# vocab_size = len(preprocess())

## Part 3
Implement the function *eval* that returns the perplexity of a model (dictionary) running over the data file of the given target language.

The `<unk>` should be used for unknown contexts when calculating the perplexities.

In [28]:
def perplexity(model: dict, text: list, n) -> float:
  '''
  Calculates the perplexity of the given string using the given language model.
  :param model: The language model
  :param text: The tokenized text to calculate the perplexity for
  :param n: The n-gram of the model
  :return: The perplexity
  '''
  pp = 0
  #######################################
  max_size = n - 1
  q = deque(['<start>'] * max_size, maxlen=max_size)
  chars = list(text)
  chars.append('<end>')
  prob_sum = 0
  prob_count = 0
  for ch in chars:
      prob_count += 1
      ngrams = ''.join(str(x) for x in q)
      token = ch
      if ngrams in model and token in model[ngrams]:
          prob = model[ngrams][token]
      else:
          # print(f"WHAT TO DOOOOOOO ngrams: {ngrams} token: {token}")
          # prob = 1 / len(preprocess())
          prob = 1 / vocab_size
      prob_sum += math.log2(prob)
      q.append(ch)
  entropy = (-1 / prob_count) * prob_sum
  pp = 2 ** entropy

  ########################################
  return pp

In [29]:
def eval(model: dict, target_lang: str, n: int) -> float:
  '''
  Return the perplexity value calculated over applying the model on the text file
  of the target_lang language.
  :param model: the language model
  :param target_lang: the target language
  :param n: The n-gram of the model
  :return: the perplexity value
  '''
  pp = 0
  ########################################
  max_size = n - 1
  csv_path = f'data/{target_lang}.csv'
  vocab_size = len(preprocess())
  prob_count = 0
  prob_sum = 0
  with open(csv_path, 'r', newline='', encoding='utf-8') as csv_file:
      reader = csv.reader(csv_file)
      next(reader, None)
      for row in reader:
          q = deque(['<start>'] * max_size, maxlen=max_size)
          chars = list(row[1])
          chars.append('<end>')
          for ch in chars:
            prob_count += 1
            ngrams = ''.join(str(x) for x in q)
            token = ch
            if ngrams in model and token in model[ngrams]:
                prob = model[ngrams][token]
            else:
                # print(f"WHAT TO DOOOOOOO ngrams: {ngrams} token: {token}")
                # prob = 1 / len(preprocess())
                prob = 1 / vocab_size
            prob_sum += math.log2(prob)
            q.append(ch)
      entropy = (-1 / prob_count) * prob_sum
      pp = 2 ** entropy
  ########################################
  return pp


In [30]:
# LM = build_lm("en", 3, True)

In [31]:
# print("Perplexity of the English 3-gram model on datasets:")
# print(f"On English: {eval(LM, 'en', 3): .2f}")
# print(f"On French: {eval(LM, 'fr', 3): .2f}")
# print(f"On Dutch: {eval(LM, 'nl', 3): .2f}")
# print(f"On Tagalog: {eval(LM, 'tl', 3): .2f}")


In [32]:
# lm1 = build_lm("en", 1, True)
# lm2 = build_lm("en", 2, True)
# lm3 = build_lm("en", 3, True)
# lm4 = build_lm("en", 4, True)

# print("Perplexity on differnet n-gram models on English")
# print(f"On 1-gram: {eval(lm1, 'en', 1): .2f}")
# print(f"On 2-gram: {eval(lm2, 'en', 2): .2f}")
# print(f"On 3-gram: {eval(lm3, 'en', 3): .2f}")
# print(f"On 4-gram: {eval(lm4, 'en', 4): .2f}")

## Part 4
Implement the *match* function that calls *eval* using a specific value of *n* for every possible language pair among the languages we have data for. You should call *eval* for every language pair four times, with each call assign a different value for *n* (1-4). Each language pair is composed of the source language and the target language. Before you make the call, you need to call the *lm* function to create the language model for the source language. Then you can call *eval* with the language model and the target language. The function should return a pandas DataFrame with the following four columns: *source_lang*, *target_lang*, *n*, *perplexity*. The values for the first two columns are the two-letter language codes. The value for *n* is the *n* you use for generating the specific perplexity values which you should store in the forth column.

In [33]:
languages = ['en', 'es', 'fr', 'in', 'it', 'nl', 'pt', 'tl']

In [34]:
from itertools import product
def match() -> pd.DataFrame:
  '''
  Return a DataFrame containing one line per every language pair and n_gram.
  Each line will contain the perplexity calculated when applying the language model
  of the source language on the text of the target language.
  :return: a DataFrame containing the perplexity values
  '''
  df  = pd.DataFrame()
  ########################################
  ngrams  = [1, 2, 3, 4]
  records = []

  # Using a list comprehension:
  triples = [
      (src_lang, target_lang, n)
      for n in ngrams
      for src_lang in languages
      for target_lang in languages
  ]
  for triple in triples:
    LM = build_lm(triple[0], triple[2], False)
    prep = eval(LM, triple[1], triple[2])
    records.append({
                'source': triple[0],
                'target': triple[1],
                'n': triple[2],
                'perplexity': prep
            })
  df = pd.DataFrame(records)
  ########################################
  return df


In [35]:
# m = match()

In [36]:
# m[:50]

## Part 5
Implement the *generate* function which takes a language code, *n*, the prompt (the starting text), the number of tokens to generate, and *r*, which is the random seed for any randomized action you plan to take in your implementation. The function should start generating tokens, one by one, using the language model of the given source language and *n*. The prompt should be used as a starting point for aligning on the probabilities to be used for generating the next token. For this exercise, you may assume a valid input in the respective language.

Note - The generation of the next token should be from the LM's distribution with NO smoothing.

In [47]:
import random
def generate(lang: str, n: int, prompt: str, number_of_tokens: int, r: int) -> str:
  '''
  Generate text in the given language using the given parameters.
  :param lang: the language of the model
  :param n: the n_gram value
  :param prompt: the prompt to start the generation
  :param number_of_tokens: the number of tokens to generate
  :param r: the random seed to use
  '''
  text = ""
  ########################################
  text = prompt
  max_size = n - 1
  LM = build_lm(lang, n, False)
  q = deque(prompt[-(max_size):], maxlen=max_size)
  random.seed(r)
  for token_iter in range(number_of_tokens):
    ngrams = ''.join(str(x) for x in q)
    options = LM[ngrams]
    # options.pop('<unk>', None)
    choices = list(options.keys())
    weights = list(options.values())
    chosen_token = random.choices(choices, weights=weights, k=1)[0]
    if chosen_token == '<end>':
      return text
    text = text + chosen_token
    q.append(chosen_token)
  ########################################
  return text

## Part 6
Play with your generate function, try to generate different texts in different language and various values of *n*. No need to submit anything of that.

In [51]:
print(generate('en', 1, "I am", 10, 5))
print(generate('en', 2, "I am", 10, 5))
print(generate('en', 3, "I am", 10, 5))
print(generate('en', 4, "I am ", 10, 5))
print(generate('es', 2, "Soy", 10, 5))
print(generate('es', 3, "Soy", 10, 5))
print(generate('fr', 2, "Je suis", 10, 5))
print(generate('fr', 3, "Je suis", 10, 5))

I amtpgLpC eLh
I amoulpeginSh
I amit: Lynmkm
I am carrive Ob
Soycalíodenye
Soy orbe
Je suis:/opapropa
Je suis tune


In [52]:
print(generate('en', 2, "I am", 20, 5))
print(generate('en', 3, "I am", 20, 5))
print(generate('en', 4, "I Love", 20, 5))
print(generate('es', 2, "Soy", 20, 5))
print(generate('es', 3, "Soy", 20, 5))
print(generate('fr', 2, "Je suis", 20, 5))
print(generate('fr', 3, "Je suis", 20, 5))

I amoulpeginShmee bie ae
I amit: Lynmkm ways.
I Love gifts @OndMade a no
Soycalíodenyegucosie ew
Soy orbe
Je suis:/opapropades tprisl
Je suis tune


# Testing

Copy the content of the **tests.py** file from the repo and paste below. This will create the results.json file and download it to your machine.

In [53]:
########################################
# PLACE TESTS HERE #
# Create tests
def test_preprocess():
    return {
        'vocab_length': len(preprocess()),
    }

def test_build_lm():
    return {
        'english_2_gram_length': len(build_lm('en', 2, True)),
        'english_3_gram_length': len(build_lm('en', 3, True)),
        'french_3_gram_length': len(build_lm('fr', 3, True)),
        'spanish_3_gram_length': len(build_lm('es', 3, True)),
    }

def test_eval():
    lm = build_lm('en', 3, True)
    return {
        'en_on_en': round(eval(lm, 'en', 3), 2),
        'en_on_fr': round(eval(lm, 'fr', 3), 2),
        'en_on_tl': round(eval(lm, 'tl', 3), 2),
        'en_on_nl': round(eval(lm, 'nl', 3), 2),
    }


def test_match():
    df = match()
    return {
        'df_shape': df.shape,
        'en_en_3': df[(df['source'] == 'en') & (df['target'] == 'en') & (df['n'] == 3)]['perplexity'].values[0],
        'en_tl_3': df[(df['source'] == 'en') & (df['target'] == 'tl') & (df['n'] == 3)]['perplexity'].values[0],
        'en_nl_3': df[(df['source'] == 'en') & (df['target'] == 'nl') & (df['n'] == 3)]['perplexity'].values[0],
    }

def test_generate():
    return {
        'english_2_gram': generate('en', 2, "I am", 20, 5),
        'english_3_gram': generate('en', 3, "I am", 20, 5),
        'english_4_gram': generate('en', 4, "I Love", 20, 5),
        'spanish_2_gram': generate('es', 2, "Soy", 20, 5),
        'spanish_3_gram': generate('es', 3, "Soy", 20, 5),
        'french_2_gram': generate('fr', 2, "Je suis", 20, 5),
        'french_3_gram': generate('fr', 3, "Je suis", 20, 5),
    }

TESTS = [test_preprocess, test_build_lm, test_eval, test_match, test_generate]

# Run tests and save results
res = {}
for test in TESTS:
    try:
        cur_res = test()
        res.update({test.__name__: cur_res})
    except Exception as e:
        res.update({test.__name__: repr(e)})

with open('results.json', 'w') as f:
    json.dump(res, f, indent=2)

# Download the results.json file
files.download('results.json')
########################################

KeyboardInterrupt: 

In [40]:
# Show the local files, results.json should be there now and
# also downloaded to your local machine
!ls -l

total 12
drwxr-xr-x 3 root root 4096 May  1 22:56 data
-rw-r--r-- 1 root root  541 May  1 23:14 results.json
drwxr-xr-x 1 root root 4096 Apr 30 13:37 sample_data
